# 0_데이터 읽어오기

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv( "/Users/luci031/Desktop/Coding/chai/data/chai_origin.csv" )
display(df.head(2))

,customer_id,created_at,pre_discount,post_discount,cashback_amount,discount_amount,total_promotion,push_permission,gender,is_foreigner,birthday,sign_up_date,merchant_id
0,207317,2020-03-22 10:27:52.498,36236,36236,1087,0,1087,True,female,False,1957-11-05,2019-11-30 16:48:19.339,9
1,95246,2020-03-12 09:41:09.845,12900,12900,387,0,387,True,female,False,1983-10-23,2019-07-09 08:21:14.098,6


# 1_컬럼명 교체하기

In [2]:
### feature rename

before = df.columns
after = "id buy pre post c_back dscnt total push man frgn birth sign m_id".split()
dic = {x:y for x,y in zip(before,after)}
print(dic)

df = df.rename( columns=dic )
df.head(2)

{'customer_id': 'id', 'created_at': 'buy', 'pre_discount': 'pre', 'post_discount': 'post', 'cashback_amount': 'c_back', 'discount_amount': 'dscnt', 'total_promotion': 'total', 'push_permission': 'push', 'gender': 'man', 'is_foreigner': 'frgn', 'birthday': 'birth', 'sign_up_date': 'sign', 'merchant_id': 'm_id'}


,id,buy,pre,post,c_back,dscnt,total,push,man,frgn,birth,sign,m_id
0,207317,2020-03-22 10:27:52.498,36236,36236,1087,0,1087,True,female,False,1957-11-05,2019-11-30 16:48:19.339,9
1,95246,2020-03-12 09:41:09.845,12900,12900,387,0,387,True,female,False,1983-10-23,2019-07-09 08:21:14.098,6


# 2_카테고리컬 컬럼(3) 전처리

In [3]:
### push(알람), frgn(외국인): True/False ---> 1/0
df["push"] = df["push"].apply(lambda x:1 if x==True else 0)
df["frgn"] = df["frgn"].apply(lambda x:1 if x==True else 0)

### man(남): male/female ---> 1/0
df["man"] = df["man"].apply(lambda x:1 if x=="male" else 0)
df.head(2)

,id,buy,pre,post,c_back,dscnt,total,push,man,frgn,birth,sign,m_id
0,207317,2020-03-22 10:27:52.498,36236,36236,1087,0,1087,1,0,0,1957-11-05,2019-11-30 16:48:19.339,9
1,95246,2020-03-12 09:41:09.845,12900,12900,387,0,387,1,0,0,1983-10-23,2019-07-09 08:21:14.098,6


# 3_나이 feature 추가

In [4]:
### age 생성 (2020년 기준 한국나이)
df[["buy","sign","birth"]] = df[["buy","sign","birth"]].astype("datetime64")
df["age"] = 2020 - df["birth"].dt.year + 1
df.head(2)

,id,buy,pre,post,c_back,dscnt,total,push,man,frgn,birth,sign,m_id,age
0,207317,2020-03-22 10:27:52.498,36236,36236,1087,0,1087,1,0,0,1957-11-05,2019-11-30 16:48:19.339,9,64
1,95246,2020-03-12 09:41:09.845,12900,12900,387,0,387,1,0,0,1983-10-23,2019-07-09 08:21:14.098,6,38


# 4_컬럼순서, 데이터순서 정렬

In [5]:
### feature 순서조절 = 고객정보 + 거래정보
df = df[['id', 'age', 'man', 'push', 'frgn', 'birth', 'sign',        # 고객 정보
        'buy', 'pre', 'post', 'c_back', 'dscnt', 'total', 'm_id' ]]  # 거래 정보

### data 순서정렬: 1순위) buy(구매시점:오름차순) --> 2순위) id(유저코드:오름차순) 순서로 정렬
df = df.sort_values( by=["buy","id"], ascending=[True,True] )
df.head(2)

,id,age,man,push,frgn,birth,sign,buy,pre,post,c_back,dscnt,total,m_id
2639466,357745,58,1,1,0,1963-03-08,2019-08-01 00:35:35.508,2019-08-01 00:32:56.331,43000,35000,0,8000,8000,9
2619610,13015,40,0,0,0,1981-08-27,2019-07-18 22:51:53.022,2019-08-01 00:32:56.346,10900,5900,0,5000,5000,7


# 5_데이터 저장

In [6]:
### parquet 포맷으로 저장
df.to_parquet( "data/df0.parquet", engine="pyarrow", compression='gzip' )

In [13]:
### 원본 읽어와서 한번에 전처리 하려면..
import numpy as np
import pandas as pd
df = pd.read_csv( "data\[이어드림스쿨] 차이코퍼레이션 데이터.csv" )
display(df.head(2))

### 완성된 전처리 함수
def pre(df):
    before = df.columns
    after = "id buy pre post c_back dscnt total push man frgn birth sign m_id".split()
    dic = {x:y for x,y in zip(before,after)}
    df = df.rename( columns=dic )
    df["push"] = df["push"].apply(lambda x:1 if x==True else 0)
    df["frgn"] = df["frgn"].apply(lambda x:1 if x==True else 0)
    df["man"] = df["man"].apply(lambda x:1 if x=="male" else 0)
    df[["buy","sign","birth"]] = df[["buy","sign","birth"]].astype("datetime64")
    df["age"] = 2020 - df["birth"].dt.year + 1
    df = df[['id', 'age', 'man', 'push', 'frgn', 'birth', 'sign',
            'buy', 'pre', 'post', 'c_back', 'dscnt', 'total', 'm_id' ]]
    df = df.sort_values( by=["buy","id"], ascending=[True,True] )
    return df

### 결과물
df = pre(df)
display(df.head(2))

,customer_id,created_at,pre_discount,post_discount,cashback_amount,discount_amount,total_promotion,push_permission,gender,is_foreigner,birthday,sign_up_date,merchant_id
0,207317,2020-03-22 10:27:52.498,36236,36236,1087,0,1087,True,female,False,1957-11-05,2019-11-30 16:48:19.339,9
1,95246,2020-03-12 09:41:09.845,12900,12900,387,0,387,True,female,False,1983-10-23,2019-07-09 08:21:14.098,6


,id,age,man,push,frgn,birth,sign,buy,pre,post,c_back,dscnt,total,m_id
2639466,357745,58,1,1,0,1963-03-08,2019-08-01 00:35:35.508,2019-08-01 00:32:56.331,43000,35000,0,8000,8000,9
2619610,13015,40,0,0,0,1981-08-27,2019-07-18 22:51:53.022,2019-08-01 00:32:56.346,10900,5900,0,5000,5000,7


# 6_(앞으로 사용할 셀)

In [14]:
import numpy as np
import pandas as pd

### 원본 읽어오기
df0 = pd.read_parquet( "new_data\df0.parquet" )
print(df0.shape)
display(df.head(2))

(5623810, 14)


,id,age,man,push,frgn,birth,sign,buy,pre,post,c_back,dscnt,total,m_id
2639466,357745,58,1,1,0,1963-03-08,2019-08-01 00:35:35.508,2019-08-01 00:32:56.331,43000,35000,0,8000,8000,9
2619610,13015,40,0,0,0,1981-08-27,2019-07-18 22:51:53.022,2019-08-01 00:32:56.346,10900,5900,0,5000,5000,7
